# Code

In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from matplotlib import pyplot
from keras.models import load_model

2022-05-26 17:52:13.326747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-26 17:52:13.326823: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
filenames = ['averaging_bb_2560.csv', 'averaging_legalbert_2560.csv', 'bb_2560.csv', 'legalbert_2560.csv', 'maxpool_bb_2560.csv', 'maxpool_legalbert_2560.csv']

In [3]:
for filename in filenames:
    data = pd.read_csv(filename)
    print(data.columns)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'label', 'uid'],
      dtype='object', length=770)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'label', 'uid'],
      dtype='object', length=770)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'uid', 'label'],
      dtype='object', length=770)
Index(['Unnamed: 0', '0', '1', '2', '3', '4', '5', '6', '7', '8',
       ...
       '759', '760', '761', '762', '763', '764', '765', '766', '767', 'label'],
      dtype='object', length=770)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760', '761', '762', '763', '764', '765', '766', '767', 'label', 'uid'],
      dtype='object', length=770)
Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '760',

In [3]:
for filename in filenames:
    data = pd.read_csv(filename)
    filename = filename[:filename.index('.')]
    label = data['label']
    features = data.drop(['uid', 'label'], axis=1)
    X, a, y, b = train_test_split(features, label, random_state=42, stratify=label, test_size=0.3) # 70% Train, 12% Train, 18% Validation
    print(X.shape, y.shape, a.shape, b.shape)

    Xv, Xt, yv, yt = train_test_split(a, b, random_state=42, stratify=b, test_size=0.4)
    print(Xv.shape, Xt.shape, yv.shape, yt.shape)

    # Xv = X[-6000:]
    # X = X[:-6000]
    print("\n\n****************** SHAPE OF DATA ****************\n\n",X.shape ,  y.shape)
    n_features = X.shape[1]
    n_features
    df = pd.DataFrame(columns=['train_acc', 'test_acc', 'batch_size'])
    for batch_size in [8, 16, 24, 32]:
      # Defining the model
      model = keras.Sequential([
          keras.layers.Dense(1000, input_shape=(n_features,), activation='relu'),
          keras.layers.Dropout(0.25),
          keras.layers.Dense(500, activation='relu'),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(200, activation='relu'),
          keras.layers.Dropout(0.125),
          keras.layers.Dense(100, activation='relu'),
          keras.layers.Dropout(0.1),
          keras.layers.Dense(1, activation='sigmoid')
      ])
      model.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(), metrics=['accuracy'])
      es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)
      mc = ModelCheckpoint(filename+'/best_model_' + filename + '_' + str(batch_size) + '_.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)
      history = model.fit(X, y, validation_data=(Xv, yv), epochs=500, batch_size=batch_size, callbacks=[es, mc])

      saved_model = load_model(filename+'/best_model_' + filename +'_' + str(batch_size) + '_.h5')
      # evaluate the model
      _, train_acc = saved_model.evaluate(X, y, verbose=0)
      _, test_acc = saved_model.evaluate(Xt, yt, verbose=0)

      lst = [train_acc, test_acc, batch_size]
      # df = df.append(lst)
      df.loc[len(df.index)] = [train_acc, test_acc, batch_size]
      print('Train: %.3f, Test: %.3f, Batch_Size: %.3f' % (train_acc, test_acc, batch_size))
      pyplot.plot(history.history['accuracy'], label='train')
      pyplot.plot(history.history['val_accuracy'], label='test')
      pyplot.savefig(filename+'/'+filename + '_' + str(batch_size) +'.png', bbox_inches='tight')
      pyplot.legend()
      pyplot.show()

    df.to_csv(filename+'/' +filename +'_New_results.csv')


(27481, 768) (27481,) (11778, 768) (11778,)
(7066, 768) (4712, 768) (7066,) (4712,)


****************** SHAPE OF DATA ****************

 (27481, 768) (27481,)


2022-05-26 06:53:48.244453: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-26 06:53:48.245521: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-26 06:53:48.245633: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-05-26 06:53:48.245764: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-05-26 06:53:48.245886: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

Epoch 1/5
3435/3436 [============================>.] - ETA: 0s - loss: 0.6887 - accuracy: 0.5635
Epoch 1: val_accuracy improved from -inf to 0.44919, saving model to averaging_bb_2560/best_model_averaging_bb_2560_8_.h5
3436/3436 [==============================] - 56s 16ms/step - loss: 0.6887 - accuracy: 0.5635 - val_loss: 0.6952 - val_accuracy: 0.4492
Epoch 2/5
2205/3436 [==================>...........] - ETA: 19s - loss: 0.6842 - accuracy: 0.5694

KeyboardInterrupt: 